In [1]:
# Installing package
!pip install gensim
!pip3 install keras
!pip install -U spacy

# Importing libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import spacy
import os
import sys

Requirement already up-to-date: spacy in /home/wangjia/anaconda3/lib/python3.7/site-packages (2.2.4)


In [ ]:
#DON'T NEED TO RUN THIS PART FOR NOW. RUN THE NEXT CELL TO LOAD DATA
#####################################################################
dataset = "Electronics_5.json"

if os.path.isfile(dataset):
    df = pd.read_json("Electronics_5.json", lines=True)
else:
    url = r"http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Electronics_5.json.gz"
    df = pd.read_json(url, compression='gzip', lines=True)

display(df.tail(10))
df.shape
print(df.info())
df_mini = df[(df.asin == "B01HJCN1EI") | (df.asin == "B01HJH42KU") | 
                            (df.asin == "B01HJH40WU") | (df.asin == "B01HJF704M") | 
                           (df.asin == "B01HJCN5GC") | (df.asin == "B01HJCN5TO") |
                           (df.asin == "B01HJDNL60") | (df.asin == "B01HJDR9DQ") |
                           (df.asin == "B01HJFFHTC") | (df.asin == "B01HJCN1EI")]
df_mini.shape
df_mini.to_csv('/home/wangjia/datasci-w266-finalProject/df_mini.csv')
######################################################################

In [27]:
df = pd.read_csv("df_mini.csv") 
display(df.tail(10))
df.shape
print(df.info())

,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
141,6739580,5,NaN,True,"07 25, 2017",A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800,NaN
142,6739581,1,NaN,True,"04 4, 2017",A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000,NaN
143,6739582,3,NaN,True,"07 8, 2017",A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000,NaN
144,6739583,2,NaN,True,"05 21, 2017",A12E1JGKV0ETAB,B01HJH42KU,NaN,John Adams,I liked the length and the product at first bu...,Lost ability to connect.,1495324800,NaN
145,6739584,3,NaN,True,"06 26, 2017",A1HKXEX8BEQC2E,B01HJH40WU,NaN,Dasha stephens,not holding up over time :(,not holding up over time :(,1498435200,NaN
146,6739585,4,NaN,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,NaN,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,NaN
147,6739586,4,NaN,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,NaN,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,NaN
148,6739587,5,2.0,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,NaN
149,6739588,5,2.0,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,NaN,Faith,Works well.,Five Stars,1480377600,NaN
150,6739589,5,NaN,True,"03 31, 2017",A1FGCIRPRNZWD5,B01HJF704M,NaN,Brando,I have it plugged into a usb extension on my g...,Works well enough..,1490918400,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 13 columns):
Unnamed: 0        151 non-null int64
overall           151 non-null int64
vote              20 non-null float64
verified          151 non-null bool
reviewTime        151 non-null object
reviewerID        151 non-null object
asin              151 non-null object
style             15 non-null object
reviewerName      151 non-null object
reviewText        151 non-null object
summary           151 non-null object
unixReviewTime    151 non-null int64
image             5 non-null object
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 14.4+ KB
None


In [15]:
#Remove NA review rows
df = df.dropna(subset=['reviewText'])
#Checking one of the reviews
print(df["reviewText"].iloc[100])

This works great, i needed a longer cable (this is 10') and stronger connection between the plug & wire.  This looks like it is stronger.


In [17]:
# Downloading stopwords
nltk.download('stopwords')

#set of stopwords in English
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
words_to_keep = set(('not'))
stop -= words_to_keep
#initialising the snowball stemmer
sno = nltk.stem.SnowballStemmer('english')

#function to clean the word of any html-tags
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

#function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wangjia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
#Code for removing HTML tags , punctuations . Code for removing stopwords . Code for checking if word is not alphanumeric and
# also greater than 2 . Code for stemmimg and also to convert them to lowercase letters 

i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in df.reviewText:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (df['reviewText'].values)[i] == 1: 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(df['reviewText'].values)[i] == 0:
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    
    
    final_string.append(str1)
    i+=1

In [32]:
#adding a column of CleanedText which displays the data after pre-processing of the review
df['CleanedText']=final_string  
df['CleanedText']=df['CleanedText'].str.decode("utf-8")
#below the processed review can be seen in the CleanedText Column 
print('Shape of final',df.shape)
df.head()

Shape of final (151, 14)


,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,CleanedText
0,6099679,5,NaN,True,"10 10, 2016",ATGTQKPUR7XIO,B01HJCN5GC,NaN,Arthur,Great buy!,Five Stars,1476057600,NaN,great buy
1,6099680,5,NaN,True,"09 8, 2016",A15VV7NPTST593,B01HJCN5GC,NaN,Randy T.,Works very well and we have lots (& lots) of e...,Extend your reach with ease,1473292800,NaN,work well lot lot extra extens abl pass devic ...
2,6099681,5,NaN,True,"03 15, 2017",AIM3MWK3Y7XOR,B01HJCN5GC,NaN,Kindle Customer,This cable is very flexible. Just what I wanted.,Flexible cable,1489536000,NaN,cabl flexibl want
3,6099682,5,NaN,True,"02 16, 2017",A5W6EI03IKOLB,B01HJCN5GC,NaN,P.Davidson,"These are the best charging cables, and if oth...",Best cables,1487203200,NaN,best charg cabl famili member didnt take would...
4,6099683,4,NaN,True,"02 14, 2017",A3QZTMHQ1XZ8PM,B01HJCN5GC,NaN,glittergirl,I bought this in rose gold or light pink and i...,super cute cord,1487030400,NaN,bought rose gold light pink clear bubblegum br...


In [26]:
#After processing the sample review looks like this
print(df["CleanedText"].iloc[100])

work great need longer cabl stronger connect plug wire look like stronger


In [34]:
#Sorting data according to asin in ascending order
sorted_data=df.sort_values('asin', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"reviewerID","reviewerName","reviewText","summary"}, keep='first', inplace=False)

#Removed not verified rows
final = final[final.verified != False]

final = final.dropna(subset=['reviewText'])
print(final.shape)
final[30:50]

(128, 14)


,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,CleanedText
8,6099687,5,NaN,True,"06 17, 2017",AY8FBAZ0J9V6Q,B01HJCN5TO,NaN,mercedes c,It charges my phone quickly. I love the length...,Works with my android,1497657600,NaN,charg phone quick love length cabl
9,6099688,5,NaN,True,"06 5, 2017",A1Q9H6HK893HTN,B01HJCN5TO,NaN,Mary D,Awesome color and love the quality. Best cord ...,Awesome!,1496620800,NaN,awesom color love qualiti best cord purchas
10,6099689,4,NaN,True,"05 12, 2017",A14XUKTQ0JLMOL,B01HJCN5TO,NaN,J. L. Wirtjes,Can't use the Kindle and have it stay plugged ...,Replacement much better than original.,1494547200,NaN,cant use kindl stay plug slightest bump seem p...
15,6099694,5,NaN,True,"12 6, 2016",AAFVMKZUY794L,B01HJCN5TO,NaN,DZ,Great heavy duty cable. With its extra length ...,Much nicer than standard Kindle charger cable,1480982400,NaN,great heavi duti cabl extra length make easi s...
19,6099698,5,NaN,True,"08 21, 2016",A37UYCR1HG32NC,B01HJCN5TO,NaN,ALW,"My daughter's favorite color is green, so this...",My daughter's favorite color is green,1471737600,NaN,daughter favorit color green perfect usb charg...
12,6099691,5,NaN,True,"03 7, 2017",A28AAIE3CHN7J9,B01HJCN5TO,NaN,Lj Anderson,The ladies asked for a review of this cord. O...,It Works. Period.,1488844800,NaN,ladi ask review cord okay make cord differ sti...
13,6099692,5,NaN,True,"02 16, 2017",A3C2ATO5SRIN31,B01HJCN5TO,NaN,k.h,real nice charging cable which fits snug,Five Stars,1487203200,NaN,real nice charg cabl fit snug
133,6739572,4,NaN,True,"10 2, 2017",A1AHM42CL4GQUA,B01HJCN5TO,NaN,Amazon Customer,"I have not used this very long,working well",Four Stars,1506902400,NaN,use long work well
18,6099697,5,NaN,True,"09 23, 2016",A1AOMNXFSEI4SZ,B01HJCN5TO,NaN,Pete Vaughn,Received cable quickly. Opened package and plu...,Long and strong,1474588800,NaN,receiv cabl quick open packag plug work perfec...
17,6099696,1,11.0,True,"10 29, 2016",A1QZ1I5MWJ6MXS,B01HJCN5TO,NaN,Chris,"Micro usb connector metal soft, easy to deform...",On well!!!,1477699200,NaN,micro usb connector metal soft easi deform bou...
